<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert TensroFlow `saved_model` to [TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/tensorflow_object_detection_api) Engine

define model location

In [1]:
workdir = '/content/workspace/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

Drive already mounted at /content/workspace; to attempt to forcibly remount, call drive.mount("/content/workspace", force_remount=True).


download `trained_model` from google drive to `workdir`

check python version

In [2]:
!python3 --version

Python 3.7.13


## Installation & Environment Configuration

install tfod api

In [3]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models /content/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

In [4]:
%%bash

# update pip
python3 -m pip install --upgrade setuptools pip

# check opencv dependency versions
python3 -m pip uninstall opencv-python-headless==4.5.5.62 
python3 -m pip install opencv-python-headless==4.1.2.30
python3 -m pip install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com

# compile protos and install object_detection
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install --use-feature=2020-resolver .

  Using cached opencv_python_headless-4.1.2.30-cp37-cp37m-manylinux1_x86_64.whl (21.8 MB)
Looking in indexes: https://pypi.ngc.nvidia.com
Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=21914838 sha256=3584e4889b9dd025abcd9e6a77e7d106771be0530c1f5277a940a1768dd4fb86
  Stored in directory: /tmp/pip-ephem-wheel-cache-aqytka0y/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [5]:
!pip3 show object-detection

Name: object-detection
Version: 0.1
Summary: Tensorflow Object Detection Library
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.7/dist-packages
Requires: apache-beam, avro-python3, contextlib2, Cython, keras, lvis, lxml, matplotlib, pandas, pillow, pycocotools, scipy, six, tensorflow-io, tf-models-official, tf-slim
Required-by: 


install tensorrt

In [6]:
!nvidia-smi

Tue May 17 22:53:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!python3 -m pip install nvidia-pyindex
!python3 -m pip install pycuda onnx==1.8.1 onnxruntime==1.8.0 tf2onnx==1.8.1
!python3 -m pip install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com
!python3 -m pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt

Looking in indexes: https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.ngc.nvidia.com


In [8]:
import tensorrt
print(tensorrt.__version__)

8.4.0.6


get tensorrt utility files

In [9]:
url = 'https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/{}.py'
files = ['build_engine', 'create_onnx', 'image_batcher', 'infer', 'compare_tf', 'infer', 'onnx_utils', 'visualize']
os.chdir('/content/utils/scripts')
for file in files:
    furl = url.format(file)
    !wget {furl}

--2022-05-17 22:53:24--  https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/build_engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12009 (12K) [text/plain]
Saving to: ‘build_engine.py.3’

build_engine.py.3   100%[===================>]  11.73K  --.-KB/s    in 0s      

2022-05-17 22:53:24 (53.6 MB/s) - ‘build_engine.py.3’ saved [12009/12009]

--2022-05-17 22:53:24--  https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/create_onnx.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connecte

## Create TensorRT Engine

In [14]:
os.chdir('/content/utils/scripts')
!echo {workdir}saved_model
!python3 create_onnx.py \
    --pipeline_config {workdir}pipeline.config \
    --saved_model {workdir}saved_model \
    --onnx {workdir}model.onnx

/content/workspace/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/saved_model
2022-05-17 23:08:06.122221: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tf2onnx.tf_loader:Signatures found in model: [serving_default].
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:ModelHelper:Loaded saved model from /content/workspace/.shortcut-targets-by-id/14CICqswNgg6A6bA_ucf0CNafkIEVmdaV/tensorflow/power-port-targeting/exported-models/trained_model/saved_model
INFO:tf2onnx.tfonnx:Using tensorflow=2.8.0, onnx=1.8.1, tf2onnx=1.8.1/4e49f3
INFO:tf2onnx.tfonnx:Using opset <onnx, 11>
INFO:tf2onnx.tf_utils:Computed 4 values for constant folding
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
INFO:tf2onnx.tf_utils:Comput

In [19]:
os.chdir('/content/utils/scripts')
!python3 build_engine.py \
    --onnx {workdir}model.onnx \
    --engine {workdir}engine.trt \
    --precision fp32 # for some reason this must be fp32, fp16 will not work

[05/17/2022-23:18:20] [TRT] [I] [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 0, GPU 112 (MiB)
[05/17/2022-23:18:21] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 0 MiB, GPU 112 MiB
[05/17/2022-23:18:21] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 0 MiB, GPU 112 MiB
build_engine.py:129: DeprecationWarning: Use set_memory_pool_limit instead.
  self.config.max_workspace_size = workspace * (2 ** 30)
[05/17/2022-23:18:22] [TRT] [W] onnx2trt_utils.cpp:365: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[05/17/2022-23:18:22] [TRT] [I] No importer registered for op: EfficientNMS_TRT. Attempting to import as plugin.
[05/17/2022-23:18:22] [TRT] [I] Searching for plugin: EfficientNMS_TRT, plugin_version: 1, plugin_namespace: 
[05/17/2022-23:18:22] [TRT] [I] Successfully created plugin: EfficientNMS_TRT
INFO:EngineBuilder:Network Description


## Test TensorRT Engine

In [12]:
os.chdir('/content/utils/scripts')
!python3 infer.py \
    --engine {workdir}engine.trt \
    --input ../img \
    --output {workdir}output \
    --preprocessor fixed_shape_resizer \
    --labels {workdir}label_map.txt

Traceback (most recent call last):
  File "infer.py", line 236, in <module>
    main(args)
  File "infer.py", line 191, in main
    with open(args.labels) as f:
FileNotFoundError: [Errno 2] No such file or directory: '/content/workspace/MyDrive/tensorflow/power-port-targeting/exported-models/trained_model/label_map.txt'


## Download TensorRT Engine (`engine.trt`)

In [13]:
os.chdir('/content/utils/scripts')
from google.colab import files
files.download(workdir + 'engine.trt')

FileNotFoundError: ignored